In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
import geopandas as gpd
import json
from shapely.geometry import Point
import os
from collections import defaultdict

In [ ]:
plt.rcParams['font.family'] = 'Times New Roman'

# 1. 读取岛屿数据
filtered_islands = pd.read_csv('../demand_get/filtered_island_1898.csv')
print(f"读取到 {len(filtered_islands)} 个岛屿")

# 2. 读取IPCC区域数据
ipcc_regions = gpd.read_file('IPCC-WGI-reference-regions-v4.geojson')
print(f"读取到 {len(ipcc_regions)} 个IPCC区域")

# 3. 将岛屿坐标映射到IPCC区域
def map_islands_to_ipcc_regions(islands_df, ipcc_regions):
    """将岛屿坐标映射到IPCC区域"""
    island_region_mapping = []
    
    for idx, island in islands_df.iterrows():
        lat, lon = island['Lat'], island['Long']
        point = Point(lon, lat)  # 注意经纬度的顺序
        
        # 查找包含该点的IPCC区域
        region_found = False
        for _, region in ipcc_regions.iterrows():
            if region.geometry.contains(point):
                island_region_mapping.append({
                    'island_lat': lat,
                    'island_lon': lon,
                    'region_name': region['Name'],
                    'region_acronym': region['Acronym'],
                    'continent': region['Continent']
                })
                region_found = True
                break
        
        if not region_found:
            island_region_mapping.append({
                'island_lat': lat,
                'island_lon': lon,
                'region_name': 'Unknown',
                'region_acronym': 'UNK',
                'continent': 'Unknown'
            })
    
    return pd.DataFrame(island_region_mapping)

print("正在将岛屿映射到IPCC区域...")
island_region_map = map_islands_to_ipcc_regions(filtered_islands, ipcc_regions)

# 4. 计算每个区域的岛屿数量
region_counts = island_region_map['region_acronym'].value_counts()
print("\\n各区域岛屿数量：")
print(region_counts.head(15))

# 5. 筛选至少有10个岛屿的区域
valid_regions = region_counts[region_counts >= 10].index.tolist()
print(f"\\n至少有10个岛屿的区域：{valid_regions}")

# 过滤数据只保留有效区域的岛屿
valid_islands = island_region_map[island_region_map['region_acronym'].isin(valid_regions)].copy()
print(f"有效岛屿数量：{len(valid_islands)}")

In [ ]:
# 6. 读取每个岛屿的需求数据并计算区域平均值
def calculate_regional_demand(valid_islands):
    """计算每个有效区域的平均冷热需求"""
    regional_demands = defaultdict(lambda: {'heating': [], 'cooling': [], 'total': []})
    
    data_path = '../demand_get/data/get1'
    missing_files = []
    
    for _, island in valid_islands.iterrows():
        lat, lon = island['island_lat'], island['island_lon']
        region_acronym = island['region_acronym']
        
        # 构建对应的需求数据文件名
        demand_file = f"demand_{lat}_{lon}.csv"
        demand_file_path = os.path.join(data_path, demand_file)
        
        if os.path.exists(demand_file_path):
            try:
                # 读取需求数据
                demand_data = pd.read_csv(demand_file_path)
                
                # 计算年度平均需求
                avg_heating = demand_data['heating_demand'].mean()
                avg_cooling = demand_data['cooling_demand'].mean() 
                avg_total = demand_data['total_demand'].mean()
                
                # 添加到对应区域
                regional_demands[region_acronym]['heating'].append(avg_heating)
                regional_demands[region_acronym]['cooling'].append(avg_cooling)
                regional_demands[region_acronym]['total'].append(avg_total)
                
            except Exception as e:
                print(f"读取文件 {demand_file} 时出错: {e}")
                missing_files.append(demand_file)
        else:
            missing_files.append(demand_file)
    
    if missing_files:
        print(f"\\n缺失 {len(missing_files)} 个需求数据文件")
    
    # 计算每个区域的平均值
    region_avg_demands = []
    for region, demands in regional_demands.items():
        if demands['heating']:  # 如果有数据
            avg_heating = np.mean(demands['heating'])
            avg_cooling = np.mean(demands['cooling'])
            avg_total = np.mean(demands['total'])
            
            region_avg_demands.append({
                'region_acronym': region,
                'avg_heating_demand': avg_heating,
                'avg_cooling_demand': avg_cooling,
                'avg_total_demand': avg_total,
                'island_count': len(demands['heating'])
            })
    
    return pd.DataFrame(region_avg_demands)

# 7. 为每个IPCC区域定义可视化坐标
def get_predefined_region_coordinates():
    """为每个IPCC区域定义固定的可视化坐标位置"""
    region_coords = {
        'CAR': (20.0, -60.0),   # Caribbean
        'NAO': (40.0, -30.0),   # North Atlantic Ocean
        'MED': (35.0, 20.0),    # Mediterranean
        'SSA': (-50.0, -70.0),  # Southern South America
        'NZ': (-40.0, 175.0),   # New Zealand
        'EAS': (30.0, 120.0),   # East Asia
        'SPO': (-40.0, -150.0), # South Pacific Ocean
        'SEA': (0.0, 125.0),    # Southeast Asia
        'NEU': (55.0, 10.0),    # Northern Europe
        'GIC': (65.0, -40.0),   # Greenland/Iceland
        'AUS': (-25.0, 135.0),  # Australia
        'SAO': (-20.0, -10.0),  # South Atlantic Ocean
        'SAS': (20.0, 80.0),    # South Asia
        'WAS': (15.0, 45.0),    # West Asia
        'CAS': (45.0, 70.0),    # Central Asia
        'ENA': (45.0, -75.0),   # Eastern North America
        'WNA': (45.0, -120.0),  # Western North America
        'CNA': (35.0, -100.0),  # Central North America
        'NEN': (65.0, 30.0),    # Northern Europe North
        'WSA': (-10.0, -60.0),  # Western South America
        'NSA': (-5.0, -55.0),   # Northern South America
        'NES': (-15.0, -45.0),  # Northeast South America
        'SAM': (-30.0, -60.0),  # South America
        'WAF': (10.0, 0.0),     # West Africa
        'CAF': (0.0, 20.0),     # Central Africa
        'EAF': (0.0, 40.0),     # East Africa
        'SAF': (-30.0, 25.0),   # Southern Africa
        'MDG': (-20.0, 47.0),   # Madagascar
        'ESB': (70.0, 120.0),   # East Siberia
        'WSB': (65.0, 80.0),    # West Siberia
        'RFE': (55.0, 135.0),   # Russian Far East
        'RAR': (75.0, 105.0),   # Russian Arctic
        'WCA': (40.0, -10.0),   # Western Central Africa
        'ECA': (50.0, 20.0),    # Eastern Central Africa
        'TIB': (32.0, 90.0),    # Tibet
        'EEU': (55.0, 40.0),    # Eastern Europe
        'SWS': (-40.0, -73.0),  # Southwest Scandinavia (修正为南美洲西南部)
        'NWS': (70.0, 15.0),    # Northwest Scandinavia
        'CEU': (50.0, 15.0),    # Central Europe
        'WCE': (45.0, 5.0),     # West Central Europe
        'ECE': (50.0, 25.0),    # East Central Europe
        'MES': (30.0, 50.0),    # Middle East South
        'MEN': (35.0, 35.0),    # Middle East North
        'ARO': (75.0, 0.0),     # Arctic Ocean
        'BOB': (15.0, 90.0),    # Bay of Bengal
        'ARS': (15.0, 50.0),    # Arabian Sea
        'SCS': (15.0, 115.0),   # South China Sea
        'IOD': (-15.0, 75.0),   # Indian Ocean Dipole
        'WIO': (-15.0, 60.0),   # Western Indian Ocean
        'EIO': (-15.0, 90.0),   # Eastern Indian Ocean
        'SIO': (-35.0, 75.0),   # Southern Indian Ocean
        'EPO': (-10.0, -120.0), # Eastern Pacific Ocean
        'NPO': (30.0, -150.0),  # North Pacific Ocean
        'ARP': (15.0, 50.0),    # Arabian Peninsula
        'SAH': (25.0, 10.0),    # Sahara
        'SCA': (10.0, -85.0),   # Southern Central America
        'NWN': (60.0, -140.0),  # Northwest North America
        'NAU': (-15.0, 135.0),  # Northern Australia
    }
    
    return region_coords

# 8. 应用预定义坐标到区域数据
def apply_predefined_coordinates(regional_demand_data):
    """为有效区域应用预定义的可视化坐标"""
    predefined_coords = get_predefined_region_coordinates()
    
    region_coords_list = []
    for _, region_data in regional_demand_data.iterrows():
        region_acronym = region_data['region_acronym']
        
        if region_acronym in predefined_coords:
            lat, lon = predefined_coords[region_acronym]
            region_coords_list.append({
                'region_acronym': region_acronym,
                'center_lat': lat,
                'center_lon': lon
            })
        else:
            # 如果没有预定义坐标，使用默认位置
            print(f"警告: 区域 {region_acronym} 没有预定义坐标，使用默认位置")
            region_coords_list.append({
                'region_acronym': region_acronym,
                'center_lat': 0.0,
                'center_lon': 0.0
            })
    
    return pd.DataFrame(region_coords_list)

print("正在计算各区域的平均需求...")
regional_demand_data = calculate_regional_demand(valid_islands)
print("\\n各区域平均需求数据：")
print(regional_demand_data)

# 应用预定义坐标
region_coordinates = apply_predefined_coordinates(regional_demand_data)

# 合并数据用于可视化
final_data = regional_demand_data.merge(region_coordinates, on='region_acronym')
print("\\n最终数据（包含预定义坐标）：")
print(final_data[['region_acronym', 'avg_heating_demand', 'avg_cooling_demand', 
                  'island_count', 'center_lat', 'center_lon']])

In [ ]:
# 9. 创建可视化图表
def create_ipcc_regional_pie_chart(final_data):
    """基于IPCC区域数据创建饼状图可视化"""
    
    # 按总需求量升序排序，确保小饼图先画，不会被大饼图覆盖
    df_sorted = final_data.sort_values(by='avg_total_demand', ascending=True)
    
    # 创建带有 cartopy 投影的图形
    fig = plt.figure(figsize=(16, 10), dpi=300)
    ax = fig.add_subplot(1, 1, 1, projection=ccrs.Robinson())
    
    # 设置背景和地图特征
    ax.set_global()
    ax.add_feature(cfeature.LAND, facecolor='#E0E0E0', zorder=0)
    ax.add_feature(cfeature.OCEAN, facecolor='#FFFFFF', zorder=0)
    ax.add_feature(cfeature.COASTLINE, linewidth=0.7, zorder=1)
    ax.add_feature(cfeature.BORDERS, linestyle=':', linewidth=0.5, alpha=0.6, zorder=1)
    
    # 定义饼图的属性
    colors = ['#7a0101', '#012f48']  # 红色代表热，蓝色代表冷
    
    # 动态调整大小的参数
    base_zoom = 0.12  # 饼图的基础缩放比例
    scale_factor = 0.00008  # 需求量对大小的影响因子
    
    # 遍历每个区域来绘制饼图
    for index, row in df_sorted.iterrows():
        lat, lon = row['center_lat'], row['center_lon']
        heating = row['avg_heating_demand']
        cooling = row['avg_cooling_demand']
        total_demand = row['avg_total_demand']
        region_name = row['region_acronym']
        island_count = row['island_count']
        
        # 如果冷热需求都为0，则跳过
        if total_demand <= 0:
            continue
        
        # 创建一个临时的、透明的画布来生成饼图图像
        fig_temp, ax_temp = plt.subplots(figsize=(2, 2), dpi=150)
        fig_temp.patch.set_alpha(0)  # 图窗背景透明
        ax_temp.patch.set_alpha(0)   # 坐标轴背景透明
        
        # 在临时坐标轴上绘制饼图
        wedges, texts = ax_temp.pie(
            [heating, cooling],
            colors=colors,
            wedgeprops={'edgecolor': 'white', 'linewidth': 1.5},
            startangle=90
        )
        ax_temp.set_aspect('equal')  # 保证是圆形
        ax_temp.axis('off')  # 关闭坐标轴显示
        fig_temp.tight_layout(pad=0)  # 去除白边
        
        # 将绘制好的饼图渲染成一个Numpy数组图像
        fig_temp.canvas.draw()
        pie_img = np.array(fig_temp.canvas.renderer.buffer_rgba())
        plt.close(fig_temp)  # 关闭临时图像，释放内存
        
        # 计算饼图的动态大小 (缩放比例)
        zoom = base_zoom + scale_factor * total_demand
        zoom = max(0.05, min(zoom, 0.3))  # 限制缩放范围
        
        # 将饼图图像作为OffsetImage添加到主地图上
        # 将地理坐标转换为地图投影坐标
        point_in_map_proj = ax.projection.transform_point(lon, lat, ccrs.Geodetic())
        
        # 创建图像盒子
        imagebox = OffsetImage(pie_img, zoom=zoom)
        imagebox.image.axes = ax
        
        # 使用AnnotationBbox将图像盒子精确放置在地图上
        ab = AnnotationBbox(
            imagebox,
            point_in_map_proj,
            frameon=False,
            pad=0
        )
        ax.add_artist(ab)
        
        # 在饼图旁边标注区域名称和岛屿数量
        label_text = f"{region_name}"
        
        # 根据位置调整标签位置
        offset_lon = 5 if lon < 0 else -5
        offset_lat = -5 if lat > 0 else 5
        
        ax.text(lon + offset_lon,
                lat + offset_lat,
                label_text,
                transform=ccrs.Geodetic(),
                ha='center',
                va='center',
                zorder=11,
                fontsize=12,
                bbox=dict(boxstyle="round,pad=0.3",
                          facecolor="white", 
                          edgecolor="gray",
                          alpha=0.8))
    
    # 创建图例
    heating_patch = mpatches.Patch(color=colors[0], label='Heating Demand')
    cooling_patch = mpatches.Patch(color=colors[1], label='Cooling Demand')
    ax.legend(handles=[heating_patch, cooling_patch], 
              loc='lower left', 
              bbox_to_anchor=(0.02, 0.02), 
              frameon=True,
              fontsize=12)
    
    # 添加标题
    # title = 'Average Heating and Cooling Demand by IPCC Regions\\n(Regions with ≥10 Islands)'
    # ax.set_title(title, fontsize=16, pad=20, weight='bold')
    
    # 移除边框
    plt.setp(ax.spines.values(), visible=False)
    
    
    plt.tight_layout()
    plt.show()
    
    return fig

# 创建可视化图表
if len(final_data) > 0:
    fig = create_ipcc_regional_pie_chart(final_data)
    print("\\n可视化完成！")
    
    # 显示统计信息
    print("\\n=== 统计信息 ===")
    print(f"有效IPCC区域数量: {len(final_data)}")
    print(f"包含的岛屿总数: {final_data['island_count'].sum()}")
    print(f"平均每区域岛屿数: {final_data['island_count'].mean():.1f}")
    print(f"最高平均总需求: {final_data['avg_total_demand'].max():.2f} kW")
    print(f"最低平均总需求: {final_data['avg_total_demand'].min():.2f} kW")
    print(f"\\n各区域详细信息:")
    for _, row in final_data.iterrows():
        print(f"{row['region_acronym']}: {row['island_count']} islands, "
              f"H={row['avg_heating_demand']:.1f}kW, C={row['avg_cooling_demand']:.1f}kW")
else:
    print("没有找到符合条件的区域数据！")

In [ ]:
# 10. 基于IPCC区域创建WT、PV、WEC容量饼图可视化

def calculate_regional_capacity(valid_islands):
    """计算每个有效区域的平均WT、PV、WEC建设容量"""
    regional_capacities = defaultdict(lambda: {'WT': [], 'PV': [], 'WEC': [], 'total': []})
    
    data_path = '../result/output_0'
    missing_files = []
    
    for _, island in valid_islands.iterrows():
        lat, lon = island['island_lat'], island['island_lon']
        region_acronym = island['region_acronym']
        
        # 构建对应的容量数据文件名
        capacity_file = f"{lat}_{lon}_capacity.csv"
        capacity_file_path = os.path.join(data_path, capacity_file)
        
        if os.path.exists(capacity_file_path):
            try:
                # 读取容量数据
                capacity_data = pd.read_csv(capacity_file_path)
                
                # 提取WT、PV、WEC容量数据
                # 数据格式: Device列包含设备名称，Optimal_Capacity列包含容量值
                wt_capacity = 0
                pv_capacity = 0
                wec_capacity = 0
                
                for _, row in capacity_data.iterrows():
                    device = row['Device'].strip()
                    capacity = float(row['Optimal_Capacity'])
                    
                    if device == 'WT':
                        wt_capacity = capacity
                    elif device == 'PV':
                        pv_capacity = capacity
                    elif device == 'WEC':
                        wec_capacity = capacity
                
                total_capacity = wt_capacity + pv_capacity + wec_capacity
                
                # 添加到对应区域
                regional_capacities[region_acronym]['WT'].append(wt_capacity)
                regional_capacities[region_acronym]['PV'].append(pv_capacity)
                regional_capacities[region_acronym]['WEC'].append(wec_capacity)
                regional_capacities[region_acronym]['total'].append(total_capacity)
                
            except Exception as e:
                print(f"读取文件 {capacity_file} 时出错: {e}")
                missing_files.append(capacity_file)
        else:
            missing_files.append(capacity_file)
    
    if missing_files:
        print(f"\\n缺失 {len(missing_files)} 个容量数据文件")
    
    # 计算每个区域的平均值
    region_avg_capacities = []
    for region, capacities in regional_capacities.items():
        if capacities['WT']:  # 如果有数据
            avg_wt = np.mean(capacities['WT'])
            avg_pv = np.mean(capacities['PV'])
            avg_wec = np.mean(capacities['WEC'])
            avg_total = np.mean(capacities['total'])
            
            region_avg_capacities.append({
                'region_acronym': region,
                'avg_wt_capacity': avg_wt,
                'avg_pv_capacity': avg_pv,
                'avg_wec_capacity': avg_wec,
                'avg_total_capacity': avg_total,
                'island_count': len(capacities['WT'])
            })
    
    return pd.DataFrame(region_avg_capacities)

def create_ipcc_regional_capacity_pie_chart(final_capacity_data):
    """基于IPCC区域数据创建WT、PV、WEC容量饼状图可视化"""
    
    # 按总容量升序排序，确保小饼图先画，不会被大饼图覆盖
    df_sorted = final_capacity_data.sort_values(by='avg_total_capacity', ascending=True)
    
    # 创建带有 cartopy 投影的图形
    fig = plt.figure(figsize=(16, 10), dpi=300)
    ax = fig.add_subplot(1, 1, 1, projection=ccrs.Robinson())
    
    # 设置背景和地图特征
    ax.set_global()
    ax.add_feature(cfeature.LAND, facecolor='#E0E0E0', zorder=0)
    ax.add_feature(cfeature.OCEAN, facecolor='#FFFFFF', zorder=0)
    ax.add_feature(cfeature.COASTLINE, linewidth=0.7, zorder=1)
    ax.add_feature(cfeature.BORDERS, linestyle=':', linewidth=0.5, alpha=0.6, zorder=1)
    
    # 定义饼图的属性 - WT、PV、WEC三种颜色
    colors = ['#cce4ef', '#daa87c', '#599cb4']  # 蓝色(WT)、橙色(PV)、深蓝色(WEC)
    
    # 动态调整大小的参数
    base_zoom = 0.12  # 饼图的基础缩放比例
    scale_factor = 0.00005  # 容量对大小的影响因子，调整以适应容量范围
    
    # 遍历每个区域来绘制饼图
    for index, row in df_sorted.iterrows():
        lat, lon = row['center_lat'], row['center_lon']
        wt_capacity = row['avg_wt_capacity']
        pv_capacity = row['avg_pv_capacity']
        wec_capacity = row['avg_wec_capacity']
        total_capacity = row['avg_total_capacity']
        region_name = row['region_acronym']
        island_count = row['island_count']
        
        # 如果总容量为0，则跳过
        if total_capacity <= 0:
            continue
        
        # 创建一个临时的、透明的画布来生成饼图图像
        fig_temp, ax_temp = plt.subplots(figsize=(2, 2), dpi=150)
        fig_temp.patch.set_alpha(0)  # 图窗背景透明
        ax_temp.patch.set_alpha(0)   # 坐标轴背景透明
        
        # 在临时坐标轴上绘制饼图
        capacities = [wt_capacity, pv_capacity, wec_capacity]
        # 过滤掉0值，避免空饼块
        non_zero_capacities = []
        non_zero_colors = []
        labels = ['WT', 'PV', 'WEC']
        
        for i, cap in enumerate(capacities):
            if cap > 0:
                non_zero_capacities.append(cap)
                non_zero_colors.append(colors[i])
        
        if non_zero_capacities:  # 如果有非零容量
            wedges, texts = ax_temp.pie(
                non_zero_capacities,
                colors=non_zero_colors,
                wedgeprops={'edgecolor': 'white', 'linewidth': 1.5},
                startangle=90
            )
        else:
            continue
            
        ax_temp.set_aspect('equal')  # 保证是圆形
        ax_temp.axis('off')  # 关闭坐标轴显示
        fig_temp.tight_layout(pad=0)  # 去除白边
        
        # 将绘制好的饼图渲染成一个Numpy数组图像
        fig_temp.canvas.draw()
        pie_img = np.array(fig_temp.canvas.renderer.buffer_rgba())
        plt.close(fig_temp)  # 关闭临时图像，释放内存
        
        # 计算饼图的动态大小 (缩放比例)
        zoom = base_zoom + scale_factor * total_capacity
        zoom = max(0.05, min(zoom, 0.25))  # 限制缩放范围
        
        # 将饼图图像作为OffsetImage添加到主地图上
        # 将地理坐标转换为地图投影坐标
        point_in_map_proj = ax.projection.transform_point(lon, lat, ccrs.Geodetic())
        
        # 创建图像盒子
        imagebox = OffsetImage(pie_img, zoom=zoom)
        imagebox.image.axes = ax
        
        # 使用AnnotationBbox将图像盒子精确放置在地图上
        ab = AnnotationBbox(
            imagebox,
            point_in_map_proj,
            frameon=False,
            pad=0
        )
        ax.add_artist(ab)
        
        # 在饼图旁边标注区域名称
        label_text = f"{region_name}"
        
        # 根据位置调整标签位置
        offset_lon = 5 if lon < 0 else -5
        offset_lat = -5 if lat > 0 else 5
        
        ax.text(lon + offset_lon,
                lat + offset_lat,
                label_text,
                transform=ccrs.Geodetic(),
                ha='center',
                va='center',
                zorder=11,
                fontsize=12,
                bbox=dict(boxstyle="round,pad=0.3",
                          facecolor="white", 
                          edgecolor="gray",
                          alpha=0.8))
    
    # 创建图例
    wt_patch = mpatches.Patch(color=colors[0], label='Wind Turbine (WT)')
    pv_patch = mpatches.Patch(color=colors[1], label='Photovoltaic (PV)')
    wec_patch = mpatches.Patch(color=colors[2], label='Wave Energy (WEC)')
    ax.legend(handles=[wt_patch, pv_patch, wec_patch], 
              loc='lower left', 
              bbox_to_anchor=(0.02, 0.02), 
              frameon=True,
              fontsize=12)
    
    # 添加标题 - 暂时注释掉
    # title = 'Average Energy Generation Capacity by IPCC Regions\\n(WT, PV, WEC - Regions with ≥10 Islands)'
    # ax.set_title(title, fontsize=16, pad=20, weight='bold')
    
    # 移除边框
    plt.setp(ax.spines.values(), visible=False)
    
    plt.tight_layout()
    plt.show()
    
    return fig

# 执行容量数据处理和可视化
print("正在计算各区域的平均WT、PV、WEC容量...")
regional_capacity_data = calculate_regional_capacity(valid_islands)

if len(regional_capacity_data) > 0:
    print("\\n各区域平均容量数据：")
    print(regional_capacity_data)
    
    # 应用预定义坐标
    capacity_coordinates = apply_predefined_coordinates(regional_capacity_data)
    
    # 合并数据用于可视化
    final_capacity_data = regional_capacity_data.merge(capacity_coordinates, on='region_acronym')
    
    print("\\n最终容量数据（包含预定义坐标）：")
    print(final_capacity_data[['region_acronym', 'avg_wt_capacity', 'avg_pv_capacity', 
                              'avg_wec_capacity', 'island_count', 'center_lat', 'center_lon']])
    
    # 创建容量可视化图表
    print("\\n正在创建IPCC区域平均容量饼状图...")
    fig_capacity = create_ipcc_regional_capacity_pie_chart(final_capacity_data)
    print("\\n容量可视化完成！")
    
    # 显示统计信息
    print("\\n=== 容量统计信息 ===")
    print(f"有效IPCC区域数量: {len(final_capacity_data)}")
    print(f"包含的岛屿总数: {final_capacity_data['island_count'].sum()}")
    print(f"平均每区域岛屿数: {final_capacity_data['island_count'].mean():.1f}")
    print(f"最高平均总容量: {final_capacity_data['avg_total_capacity'].max():.2f} MW")
    print(f"最低平均总容量: {final_capacity_data['avg_total_capacity'].min():.2f} MW")
    print(f"\\n各区域容量详细信息:")
    for _, row in final_capacity_data.iterrows():
        print(f"{row['region_acronym']}: {row['island_count']} islands, "
              f"WT={row['avg_wt_capacity']:.1f}MW, PV={row['avg_pv_capacity']:.1f}MW, WEC={row['avg_wec_capacity']:.1f}MW")
else:
    print("没有找到符合条件的区域容量数据！")

In [ ]:
# 11. 基于IPCC区域的季节性能源输出分析

def analyze_regional_seasonal_outputs(valid_islands, resource_type='PV'):
    """为每个IPCC区域分析季节性能源输出变化
    
    参数:
    valid_islands - 有效岛屿数据框
    resource_type - 要分析的资源类型 ('PV', 'WT', 'WEC')
    
    修正：使用3小时时间分辨率，总共2929个时间步
    """
    print(f"正在分析 {resource_type} 的季节性输出变化...")
    
    output_folder = '../result/output_0'
    regional_seasonal_data = defaultdict(lambda: {'spring': [], 'summer': [], 'autumn': [], 'winter': []})
    
    processed_islands = 0
    missing_files = []
    
    # 遍历每个有效岛屿
    for _, island in valid_islands.iterrows():
        lat = island['island_lat']
        lon = island['island_lon']
        region_acronym = island['region_acronym']
        
        # 构建对应的输出文件名
        output_file = os.path.join(output_folder, f"{lat}_{lon}_results.csv")
        
        if os.path.exists(output_file):
            try:
                # 读取数据
                system_output = pd.read_csv(output_file)
                
                # 确保所需资源存在于数据中
                if resource_type not in system_output.columns:
                    print(f"警告: {output_file} 中不存在 {resource_type} 列")
                    continue
                    
                # 提取资源数据
                resource_data = system_output[resource_type]
                
                # 使用3小时分辨率，从2020年1月1日开始
                # 2929个时间步 × 3小时 = 8787小时 ≈ 365.25天
                start_date = '2020-01-01'
                time_steps = len(resource_data)
                
                # 创建3小时间隔的时间索引
                hours = pd.date_range(start=start_date, periods=time_steps, freq='3h')
                df = pd.DataFrame({'output': resource_data.values}, index=hours)
                
                # 根据半球定义季节
                is_northern = lat > 0
                
                # 为每个时间点分配季节
                df['month'] = df.index.month
                if is_northern:
                    # 北半球季节划分
                    df['season'] = 'spring'  # 初始化
                    df.loc[df['month'].isin([12, 1, 2]), 'season'] = 'winter'
                    df.loc[df['month'].isin([6, 7, 8]), 'season'] = 'summer'
                    df.loc[df['month'].isin([3, 4, 5]), 'season'] = 'spring'
                    df.loc[df['month'].isin([9, 10, 11]), 'season'] = 'autumn'
                else:
                    # 南半球季节划分
                    df['season'] = 'spring'  # 初始化
                    df.loc[df['month'].isin([6, 7, 8]), 'season'] = 'winter'
                    df.loc[df['month'].isin([12, 1, 2]), 'season'] = 'summer'
                    df.loc[df['month'].isin([9, 10, 11]), 'season'] = 'spring'
                    df.loc[df['month'].isin([3, 4, 5]), 'season'] = 'autumn'
                
                # 计算每日平均输出量（每天8个时间步，因为24小时/3小时=8）
                df['date'] = df.index.date
                daily_output = df.groupby(['date', 'season'])['output'].mean().reset_index()
                
                # 将数据添加到对应区域
                for season in ['spring', 'summer', 'autumn', 'winter']:
                    season_data = daily_output[daily_output['season'] == season]['output'].tolist()
                    if season_data:  # 只有当季节有数据时才添加
                        regional_seasonal_data[region_acronym][season].extend(season_data)
                
                processed_islands += 1
                
            except Exception as e:
                print(f"处理 {output_file} 时出错: {e}")
                missing_files.append(output_file)
        else:
            missing_files.append(output_file)
    
    print(f"成功处理了 {processed_islands} 个岛屿的数据")
    if missing_files:
        print(f"缺失 {len(missing_files)} 个输出文件")
    
    # 计算每个区域的季节性统计
    regional_seasonal_stats = []
    for region, seasonal_data in regional_seasonal_data.items():
        # 检查是否至少有两个季节的数据
        seasons_with_data = [season for season in ['spring', 'summer', 'autumn', 'winter'] 
                           if seasonal_data[season]]
        
        if len(seasons_with_data) >= 2:  # 至少需要两个季节的数据
            # 计算季节平均值
            season_averages = {}
            for season in ['spring', 'summer', 'autumn', 'winter']:
                if seasonal_data[season]:
                    season_averages[season] = np.mean(seasonal_data[season])
                else:
                    season_averages[season] = 0  # 无数据的季节设为0
            
            # 计算季节变异性指标
            valid_averages = [v for v in season_averages.values() if v > 0]
            
            summer_avg = season_averages.get('summer', 0)
            winter_avg = season_averages.get('winter', 0)
            
            if summer_avg > 0:
                winter_summer_ratio = (winter_avg / summer_avg) * 100
            else:
                winter_summer_ratio = 0
            
            regional_seasonal_stats.append({
                'region_acronym': region,
                'spring_avg': season_averages['spring'],
                'summer_avg': season_averages['summer'],
                'autumn_avg': season_averages['autumn'],
                'winter_avg': season_averages['winter'],
                'winter_summer_ratio': winter_summer_ratio,
                'seasonal_variability': np.std(list(season_averages.values())) if len(valid_averages) > 1 else 0,
                'seasons_available': len(seasons_with_data)
            })
    
    regional_seasonal_df = pd.DataFrame(regional_seasonal_stats)
    
    if len(regional_seasonal_df) > 0:
        print(f"\n=== {resource_type} 季节性分析结果（3小时分辨率）===")
        print(f"有效区域数: {len(regional_seasonal_df)}")
        print(regional_seasonal_df[['region_acronym', 'spring_avg', 'summer_avg', 'autumn_avg', 'winter_avg', 'seasons_available']].round(2))
        
        # 创建Nature风格的季节性热图
        plt.rcParams['font.size'] = 10
        
        fig, ax = plt.subplots(figsize=(14, 6), dpi=300)
        
        # 准备热图数据
        heatmap_data = regional_seasonal_df[['spring_avg', 'summer_avg', 'autumn_avg', 'winter_avg']].values.T
        
        # 使用科学色带
        import matplotlib.colors as mcolors
        
        # 创建colormap
        colors_list = ['#ffffff', '#bbe6fa', '#4596cd', '#015696', '#ee9d9f', '#c84747', '#982b2d']
        n_bins = 256
        cmap = mcolors.LinearSegmentedColormap.from_list('sci_with_white', colors_list, N=n_bins)
        
        # 处理数据范围
        data_min = np.min(heatmap_data)
        data_max = np.max(heatmap_data)
        
        if data_max > 0:
            # 创建热图
            im = ax.imshow(heatmap_data, cmap=cmap, aspect='auto', 
                          interpolation='nearest', vmin=0, vmax=data_max)
            
            # 设置坐标轴标签
            ax.set_xticks(range(len(regional_seasonal_df)))
            ax.set_xticklabels(regional_seasonal_df['region_acronym'], 
                              fontsize=10, rotation=45, ha='right')
            ax.set_yticks(range(4))
            ax.set_yticklabels(['Spring', 'Summer', 'Autumn', 'Winter'], 
                              fontsize=11, rotation=0, va='center')
            
            # 在每个格子中添加数值标注
            for i in range(4):  # seasons
                for j in range(len(regional_seasonal_df)):  # regions
                    actual_value = heatmap_data[i, j]
                    if actual_value == 0:
                        text = '0'
                        text_color = 'gray'
                    else:
                        text = f'{actual_value:.1f}'
                        # 根据数值大小选择文字颜色
                        text_color = 'white' if actual_value > data_max * 0.5 else 'black'
                        
                    ax.text(j, i, text, 
                           ha='center', va='center', 
                           fontsize=8, fontweight='bold',
                           color=text_color)
            
            # 设置标题和标签
            ax.set_xlabel('IPCC Regions', fontsize=12, fontweight='bold', labelpad=15)
            ax.set_ylabel('Seasons', fontsize=12, fontweight='bold', labelpad=15)
            
            # 添加颜色条
            cbar = plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04, aspect=30)
            cbar.set_label(f'{resource_type} Daily Average Output (MW)', 
                          fontsize=11, fontweight='bold', rotation=270, labelpad=20)
            cbar.ax.tick_params(labelsize=9)
            
            # 设置网格线
            ax.set_xticks(np.arange(-0.5, len(regional_seasonal_df), 1), minor=True)
            ax.set_yticks(np.arange(-0.5, 4, 1), minor=True)
            ax.grid(which='minor', color='lightgray', linestyle='-', linewidth=1)
            
            # 移除外边框
            for spine in ax.spines.values():
                spine.set_visible(False)
            
            # 设置刻度线样式
            ax.tick_params(which='major', length=0)
            ax.tick_params(which='minor', length=0)
            
            # 调整布局
            plt.tight_layout()
            
            # 添加图注
            fig.text(0.02, 0.02, f'Seasonal {resource_type} output variation (3-hour resolution, white=no output)', 
                    fontsize=9, style='italic', alpha=0.7)
            
            plt.show()
        
        return regional_seasonal_df
    else:
        print(f"没有找到 {resource_type} 的有效季节性数据")
        return None

# 运行季节性分析（如果有有效岛屿数据）
if 'valid_islands' in locals() and len(valid_islands) > 0:
    print("\n开始进行季节性能源输出分析（使用3小时时间分辨率）...")
    
    # 分析季节性输出
    pv_seasonal_results = analyze_regional_seasonal_outputs(valid_islands, 'PV')
    wt_seasonal_results = analyze_regional_seasonal_outputs(valid_islands, 'WT')
    wec_seasonal_results = analyze_regional_seasonal_outputs(valid_islands, 'WEC')
    
else:
    print("请先运行前面的代码来生成 valid_islands 数据")

In [ ]:
# 12. 创建不同资源类型季节性比值差异散点图

def create_seasonal_ratio_scatter_plot(pv_results, wt_results, wec_results):
    """
    创建散点图展示不同资源类型的季节性比值差异
    X轴：区域（IPCC区域名称）
    Y轴：冬夏比值（Winter/Summer ratio）
    不同颜色和形状表示不同的资源类型
    """
    plt.rcParams['font.family'] = 'Times New Roman'
    plt.rcParams['font.size'] = 14
    
    # 创建图形
    fig, ax = plt.subplots(figsize=(14, 6), dpi=300)
    
    # 定义资源类型的颜色和标记
    resource_styles = {
        'PV': {'color': '#daa87c', 'marker': 'o', 'edgecolor': 'black', 'size': 130, 'alpha': 1, 'label': 'Photovoltaic (PV)'},
        'WT': {'color': '#cce4ef', 'marker': '^', 'edgecolor': 'black', 'size': 130, 'alpha': 1, 'label': 'Wind Turbine (WT)'},
        'WEC': {'color': '#599cb4', 'marker': 's', 'edgecolor': 'black', 'size': 130, 'alpha': 1, 'label': 'Wave Energy (WEC)'}
    }

    # 收集所有区域名称
    all_regions = set()
    plot_data = []
    
    # 处理PV数据
    if pv_results is not None and len(pv_results) > 0:
        for _, row in pv_results.iterrows():
            region = row['region_acronym']
            ratio = row['winter_summer_ratio']
            all_regions.add(region)
            plot_data.append({
                'resource': 'PV',
                'region': region,
                'ratio': ratio,
                'summer_avg': row['summer_avg'],
                'winter_avg': row['winter_avg']
            })
    
    # 处理WT数据
    if wt_results is not None and len(wt_results) > 0:
        for _, row in wt_results.iterrows():
            region = row['region_acronym']
            ratio = row['winter_summer_ratio']
            all_regions.add(region)
            plot_data.append({
                'resource': 'WT',
                'region': region,
                'ratio': ratio,
                'summer_avg': row['summer_avg'],
                'winter_avg': row['winter_avg']
            })
    
    # 处理WEC数据
    if wec_results is not None and len(wec_results) > 0:
        for _, row in wec_results.iterrows():
            region = row['region_acronym']
            ratio = row['winter_summer_ratio']
            all_regions.add(region)
            plot_data.append({
                'resource': 'WEC',
                'region': region,
                'ratio': ratio,
                'summer_avg': row['summer_avg'],
                'winter_avg': row['winter_avg']
            })
    
    # 将数据转换为DataFrame便于处理
    plot_df = pd.DataFrame(plot_data)
    
    if len(plot_df) == 0:
        print("没有可用于绘制散点图的数据")
        return None
    
    print(f"绘制散点图数据点数量: {len(plot_df)}")
    
    # 按区域排序（按区域名称字母顺序）
    sorted_regions = sorted(all_regions)
    
    # 为每个区域分配x轴位置
    region_positions = {region: i for i, region in enumerate(sorted_regions)}
    
    # 按资源类型分组绘制散点图
    for resource_type in ['PV', 'WT', 'WEC']:
        resource_data = plot_df[plot_df['resource'] == resource_type]
        
        if len(resource_data) > 0:
            style = resource_styles[resource_type]
            
            # 计算x轴位置，对同一区域的不同资源稍微错开
            x_positions = []
            offset_map = {'PV': -0.1, 'WT': 0.0, 'WEC': 0.1}
            
            for _, row in resource_data.iterrows():
                x_pos = region_positions[row['region']] + offset_map[resource_type]
                x_positions.append(x_pos)
            
            scatter = ax.scatter(
                x_positions,
                resource_data['ratio'],
                c=style['color'],
                marker=style['marker'],
                s=style['size'],
                alpha=style['alpha'],
                label=style['label'],
                edgecolors=style['edgecolor'],
                linewidth=1
            )
    
    # 添加水平参考线（100%表示冬夏输出相等）
    ax.axhline(y=100, color='gray', linestyle='-', alpha=0.3, linewidth=1, zorder=0)
    ax.text(0.02, 0.95, 'Winter = Summer (100%)', transform=ax.transAxes, 
           fontsize=10, alpha=0.6, style='italic')
    
    # 设置坐标轴
    ax.set_xlabel('IPCC Regions', fontsize=13, fontweight='bold')
    ax.set_ylabel('Winter/Summer Output Ratio (%)', fontsize=13, fontweight='bold')
    
    # 设置x轴刻度和标签
    ax.set_xlim(-0.5, len(sorted_regions) - 0.5)
    ax.set_xticks(range(len(sorted_regions)))
    ax.set_xticklabels(sorted_regions, rotation=45, ha='right')
    
    # 设置y轴范围
    ax.set_ylim(0, max(plot_df['ratio'].max() * 1.2, 200))
    
    # 添加网格
    ax.grid(True, alpha=0.3, linestyle='-', linewidth=0.5, axis='y')
    
    # 设置图例
    legend = ax.legend(loc='upper right', frameon=True, framealpha=0.9, 
                      fancybox=True, shadow=True, fontsize=11)
    legend.get_frame().set_facecolor('white')
    legend.get_frame().set_edgecolor('gray')
    
    # 设置坐标轴样式
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_linewidth(1.2)
    ax.spines['bottom'].set_linewidth(1.2)
    
    # 设置刻度
    ax.tick_params(axis='both', which='major', labelsize=10, length=6, width=1.2)
    ax.tick_params(axis='x', rotation=45)
    
    # 设置背景色
    ax.set_facecolor('#FAFAFA')
    
    # 调整布局
    plt.tight_layout()
    
    # 添加图注
    fig.text(0.02, 0.02, 
            'Seasonal output ratio variation across IPCC regions for different renewable energy resources',
            fontsize=10, style='italic', alpha=0.7)
    
    plt.show()
    
    # 输出统计信息
    print("\n=== 季节性比值统计信息 ===")
    for resource_type in ['PV', 'WT', 'WEC']:
        resource_data = plot_df[plot_df['resource'] == resource_type]
        if len(resource_data) > 0:
            print(f"{resource_type}:")
            print(f"  平均冬夏比值: {resource_data['ratio'].mean():.1f}%")
            print(f"  比值标准差: {resource_data['ratio'].std():.1f}%")
            print(f"  最高比值区域: {resource_data.loc[resource_data['ratio'].idxmax(), 'region']} ({resource_data['ratio'].max():.1f}%)")
            print(f"  最低比值区域: {resource_data.loc[resource_data['ratio'].idxmin(), 'region']} ({resource_data['ratio'].min():.1f}%)")
    
    return fig, plot_df

# 如果前面的季节性分析已经运行，则创建散点图
if 'pv_seasonal_results' in locals() or 'wt_seasonal_results' in locals() or 'wec_seasonal_results' in locals():
    print("\n正在创建季节性比值差异散点图...")
    
    # 确保变量存在，如果不存在则设为None
    pv_results = pv_seasonal_results if 'pv_seasonal_results' in locals() else None
    wt_results = wt_seasonal_results if 'wt_seasonal_results' in locals() else None  
    wec_results = wec_seasonal_results if 'wec_seasonal_results' in locals() else None
    
    fig_scatter, scatter_data = create_seasonal_ratio_scatter_plot(pv_results, wt_results, wec_results)
    
    if fig_scatter is not None:
        print("\n季节性比值差异散点图创建完成！")
    else:
        print("散点图创建失败，请检查数据")
        
else:
    print("请先运行季节性分析代码以生成必要的数据")